In [1]:
import pandas as pd # For dataframes (tables)
import numpy as np # For arange
import matplotlib.pyplot as plt # For plots
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import math # for sqrt
from sklearn.model_selection import cross_val_score, KFold
import pickle # to serialize/deserialize objects

from statistics import mean


In [2]:
!wget https://raw.githubusercontent.com/andreaaraldo/machine-learning-for-networks/master/course_library/andrea_models.py

from andrea_models import AndreaLinearRegression # In the course library

--2022-02-01 08:26:10--  https://raw.githubusercontent.com/andreaaraldo/machine-learning-for-networks/master/course_library/andrea_models.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3190 (3.1K) [text/plain]
Saving to: ‘andrea_models.py’

andrea_models.py    100%[===================>]   3.12K  --.-KB/s    in 0s      

2022-02-01 08:26:11 (38.1 MB/s) - ‘andrea_models.py’ saved [3190/3190]



/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
path = "https://raw.githubusercontent.com/Werner-DB/ML_Project/main/PreProcessing/dataset_cleaned.csv" # path for the preprocessed datadet
!wget $path

--2022-02-01 08:26:12--  https://raw.githubusercontent.com/Werner-DB/ML_Project/main/PreProcessing/dataset_cleaned.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3429217 (3.3M) [text/plain]
Saving to: ‘dataset_cleaned.csv’

dataset_cleaned.csv 100%[===================>]   3.27M  --.-KB/s    in 0.07s   

2022-02-01 08:26:13 (46.8 MB/s) - ‘dataset_cleaned.csv’ saved [3429217/3429217]



In [4]:
df = pd.read_csv("dataset_cleaned.csv", index_col="PROC_TRACEINFO")

In [5]:
df.head()

,OP070_V_1_angle_value,OP090_SnapRingPeakForce_value,OP070_V_2_angle_value,OP120_Rodage_I_mesure_value,OP090_SnapRingFinalStroke_value,OP110_Vissage_M8_torque_value,OP120_Rodage_U_mesure_value,OP070_V_1_torque_value,OP090_StartLinePeakForce_value,OP110_Vissage_M8_angle_value,OP090_SnapRingMidPointForce_val,OP070_V_2_torque_value,results
PROC_TRACEINFO,,,,,,,,,,,,,
I-B-XA1207672-190429-00688,180.4,190.51,173.1,113.64,12.04,12.16,11.97,6.62,26.37,18.8,109.62,6.60,0
I-B-XA1207672-190828-00973,138.7,147.70,163.5,109.77,12.12,12.19,11.97,6.41,21.03,18.5,105.48,6.40,0
I-B-XA1207672-190712-03462,180.9,150.87,181.2,109.79,11.86,12.24,11.97,6.62,25.81,17.5,100.03,6.61,0
I-B-XA1207672-190803-00051,173.5,159.56,151.8,113.25,11.82,12.35,11.97,6.62,24.62,15.6,104.94,6.61,0
I-B-XA1207672-190508-03248,174.5,172.29,177.5,112.88,12.07,12.19,11.97,6.62,29.22,33.6,99.19,6.61,0


# Train-test-split

In [6]:
df_train, df_test = train_test_split(df, test_size=0.3, random_state=19)
df_test.head()

print('Training samples ', df_train.shape[0], '; Test samples ', df_test.shape[0])

Training samples  24160 ; Test samples  10355


# Multivariate Linear Regression

In our dataset we do not have continuous values ​​as a target, so it is not recommended to apply regression, however, we do it with the intention of testing all methods.

We did not find a defined correlation of the independent variables of the dataset with the true label column that we want to predict: the results column. For this reason we will use multivariate regression including all the independent variables.

In [7]:
cols = ['OP070_V_1_angle_value', 'OP090_SnapRingPeakForce_value',
       'OP070_V_2_angle_value', 'OP120_Rodage_I_mesure_value',
       'OP090_SnapRingFinalStroke_value', 'OP110_Vissage_M8_torque_value',
       'OP120_Rodage_U_mesure_value', 'OP070_V_1_torque_value',
       'OP090_StartLinePeakForce_value', 'OP110_Vissage_M8_angle_value',
       'OP090_SnapRingMidPointForce_val', 'OP070_V_2_torque_value']
X_train = df_train[cols].values
y_train = df_train['results'].values
model = AndreaLinearRegression()
model.fit(X_train, y_train, column_names=cols)
model.summary()

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     8.347
Date:                Tue, 01 Feb 2022   Prob (F-statistic):           5.63e-16
Time:                        08:26:14   Log-Likelihood:                 22319.
No. Observations:               24160   AIC:                        -4.461e+04
Df Residuals:                   24147   BIC:                        -4.451e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
intercept 

In [8]:
X_test = df_test[cols].values
y_test = df_test['results'].values
y_pred = model.predict(X_test)

In [9]:
MSE = mean_squared_error(y_test, y_pred)
RMSE = math.sqrt(MSE)
print('RMSE: ', RMSE)

RMSE:  0.08692032895730477


# Cross-validation



In [10]:
X = df[cols].values
y = df['results'].values


k_fold = KFold(n_splits=5, shuffle=True, random_state=6)

scores = cross_val_score(AndreaLinearRegression(),
                X, y, cv=k_fold,
                scoring='neg_mean_squared_error')
scores

array([-0.00720136, -0.00842696, -0.00999997, -0.00973272, -0.00830639])

In [11]:
mse_vector = -scores
mse_vector

array([0.00720136, 0.00842696, 0.00999997, 0.00973272, 0.00830639])

In [12]:
avg_mse = mean(mse_vector)
avg_mse

0.008733478420407557

In [13]:
math.sqrt(avg_mse)

0.09345308138530027

When performing the cross validation, we see that the average of the RMSE is more than the RMSE that we had obtained when doing the multivariate regression.